<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-таблицу-и-изучите-общую-информацию-о-данных" data-toc-modified-id="Откройте-таблицу-и-изучите-общую-информацию-о-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте таблицу и изучите общую информацию о данных</a></span><ul class="toc-item"><li><span><a href="#Задание-1.-Импортируйте-библиотеку-pandas.-Считайте-данные-из-csv-файла-в-датафрейм-и-сохраните-в-переменную-data.-Путь-к-файлу:" data-toc-modified-id="Задание-1.-Импортируйте-библиотеку-pandas.-Считайте-данные-из-csv-файла-в-датафрейм-и-сохраните-в-переменную-data.-Путь-к-файлу:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную data. Путь к файлу:</a></span></li><li><span><a href="#Задание-2.-Выведите-первые-20-строчек-датафрейма-data-на-экран." data-toc-modified-id="Задание-2.-Выведите-первые-20-строчек-датафрейма-data-на-экран.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Задание 2. Выведите первые 20 строчек датафрейма data на экран.</a></span></li><li><span><a href="#Задание-3.-Выведите-основную-информацию-о-датафрейме-с-помощью-метода-info()." data-toc-modified-id="Задание-3.-Выведите-основную-информацию-о-датафрейме-с-помощью-метода-info().-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Задание 3. Выведите основную информацию о датафрейме с помощью метода info().</a></span></li></ul></li><li><span><a href="#Предобработка-данных¶" data-toc-modified-id="Предобработка-данных¶-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных¶</a></span><ul class="toc-item"><li><span><a href="#Удаление-пропусков" data-toc-modified-id="Удаление-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Удаление пропусков</a></span><ul class="toc-item"><li><span><a href="#Задание-4.-Выведите-количество-пропущенных-значений-для-каждого-столбца.-Используйте-комбинацию-двух-методов." data-toc-modified-id="Задание-4.-Выведите-количество-пропущенных-значений-для-каждого-столбца.-Используйте-комбинацию-двух-методов.-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.</a></span></li><li><span><a href="#Задание-5.-В-двух-столбцах-есть-пропущенные-значения." data-toc-modified-id="Задание-5.-В-двух-столбцах-есть-пропущенные-значения.-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Задание 5. В двух столбцах есть пропущенные значения.</a></span></li></ul></li><li><span><a href="#Обработка-аномальных-значений" data-toc-modified-id="Обработка-аномальных-значений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обработка аномальных значений</a></span><ul class="toc-item"><li><span><a href="#Задание-6.-В-данных-могут-встречаться-артефакты-(аномалии)" data-toc-modified-id="Задание-6.-В-данных-могут-встречаться-артефакты-(аномалии)-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Задание 6. В данных могут встречаться артефакты (аномалии)</a></span></li><li><span><a href="#Задание-7.-Для-каждого-типа-занятости-выведите-медианное-значение-трудового-стажа-days_employed-в-днях." data-toc-modified-id="Задание-7.-Для-каждого-типа-занятости-выведите-медианное-значение-трудового-стажа-days_employed-в-днях.-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа days_employed в днях.</a></span></li><li><span><a href="#Задание-8.-Выведите-перечень-уникальных-значений-столбца-children." data-toc-modified-id="Задание-8.-Выведите-перечень-уникальных-значений-столбца-children.-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Задание 8. Выведите перечень уникальных значений столбца children.</a></span></li><li><span><a href="#Задание-9.-В-столбце-children-есть-два-аномальных-значения." data-toc-modified-id="Задание-9.-В-столбце-children-есть-два-аномальных-значения.-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Задание 9. В столбце children есть два аномальных значения.</a></span></li><li><span><a href="#Задание-10." data-toc-modified-id="Задание-10.-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>Задание 10.</a></span></li></ul></li><li><span><a href="#Удаление-пропусков-(продолжение)" data-toc-modified-id="Удаление-пропусков-(продолжение)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Удаление пропусков (продолжение)</a></span><ul class="toc-item"><li><span><a href="#Задание-11.-Заполните-пропуски-в-столбце-days_employed-медианными-значениями-по-каждому-типу-занятости-income_type." data-toc-modified-id="Задание-11.-Заполните-пропуски-в-столбце-days_employed-медианными-значениями-по-каждому-типу-занятости-income_type.-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Задание 11. Заполните пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type.</a></span></li><li><span><a href="#Задание-12.-Убедитесь,-что-все-пропуски-заполнены." data-toc-modified-id="Задание-12.-Убедитесь,-что-все-пропуски-заполнены.-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Задание 12. Убедитесь, что все пропуски заполнены.</a></span></li></ul></li><li><span><a href="#Изменение-типов-данных" data-toc-modified-id="Изменение-типов-данных-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Изменение типов данных</a></span><ul class="toc-item"><li><span><a href="#Задание-13.-Замените-вещественный-тип-данных-в-столбце-total_income-на-целочисленный-с-помощью-метода-astype()." data-toc-modified-id="Задание-13.-Замените-вещественный-тип-данных-в-столбце-total_income-на-целочисленный-с-помощью-метода-astype().-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Задание 13. Замените вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().</a></span></li></ul></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Обработка дубликатов</a></span><ul class="toc-item"><li><span><a href="#Задание-14.-Обработайте-неявные-дубликаты-в-столбце-education." data-toc-modified-id="Задание-14.-Обработайте-неявные-дубликаты-в-столбце-education.-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>Задание 14. Обработайте неявные дубликаты в столбце education.</a></span></li><li><span><a href="#Задание-15.-Выведите-на-экран-количество-строк-дубликатов-в-данных." data-toc-modified-id="Задание-15.-Выведите-на-экран-количество-строк-дубликатов-в-данных.-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span>Задание 15. Выведите на экран количество строк-дубликатов в данных.</a></span></li></ul></li><li><span><a href="#2.6--Категоризация-данных" data-toc-modified-id="2.6--Категоризация-данных-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>2.6  Категоризация данных</a></span><ul class="toc-item"><li><span><a href="#Задание-16.-На-основании-диапазонов,-указанных-ниже,-создайте-в-датафрейме-data-столбец-total_income_category-с-категориями:" data-toc-modified-id="Задание-16.-На-основании-диапазонов,-указанных-ниже,-создайте-в-датафрейме-data-столбец-total_income_category-с-категориями:-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме data столбец total_income_category с категориями:</a></span></li><li><span><a href="#Задание-17.-Выведите-на-экран-перечень-уникальных-целей-взятия-кредита-из-столбца-purpose." data-toc-modified-id="Задание-17.-Выведите-на-экран-перечень-уникальных-целей-взятия-кредита-из-столбца-purpose.-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца purpose.</a></span></li><li><span><a href="#Задание-18.-Создайте-функцию,-которая-на-основании-данных-из-столбца-purpose-сформирует-новый-столбец-purpose_category," data-toc-modified-id="Задание-18.-Создайте-функцию,-которая-на-основании-данных-из-столбца-purpose-сформирует-новый-столбец-purpose_category,-2.6.3"><span class="toc-item-num">2.6.3&nbsp;&nbsp;</span>Задание 18. Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category,</a></span></li></ul></li></ul></li><li><span><a href="#Вторая-часть" data-toc-modified-id="Вторая-часть-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вторая часть</a></span><ul class="toc-item"><li><span><a href="#3.-Исследуйте-данные-и-ответьте-на-вопросы" data-toc-modified-id="3.-Исследуйте-данные-и-ответьте-на-вопросы-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>3. Исследуйте данные и ответьте на вопросы</a></span><ul class="toc-item"><li><span><a href="#Задание-19.-Есть-ли-зависимость-между-количеством-детей-и-возвратом-кредита-в-срок?" data-toc-modified-id="Задание-19.-Есть-ли-зависимость-между-количеством-детей-и-возвратом-кредита-в-срок?-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?</a></span></li><li><span><a href="#Задание-20.-Есть-ли-зависимость-между-семейным-положением-и-возвратом-кредита-в-срок?" data-toc-modified-id="Задание-20.-Есть-ли-зависимость-между-семейным-положением-и-возвратом-кредита-в-срок?-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?</a></span></li><li><span><a href="#Задание-21.-Есть-ли-зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок?" data-toc-modified-id="Задание-21.-Есть-ли-зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок?-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</a></span></li><li><span><a href="#Задание-22.-Как-разные-цели-кредита-влияют-на-его-возврат-в-срок?" data-toc-modified-id="Задание-22.-Как-разные-цели-кредита-влияют-на-его-возврат-в-срок?-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Задание 22. Как разные цели кредита влияют на его возврат в срок?</a></span></li><li><span><a href="#Задание-23.-Приведите-возможные-причины-появления-пропусков-в-исходных-данных." data-toc-modified-id="Задание-23.-Приведите-возможные-причины-появления-пропусков-в-исходных-данных.-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Задание 23. Приведите возможные причины появления пропусков в исходных данных.</a></span></li><li><span><a href="#Задание-24.-Объясните,-почему-заполнить-пропуски-медианным-значением-—-лучшее-решение-для-количественных-переменных." data-toc-modified-id="Задание-24.-Объясните,-почему-заполнить-пропуски-медианным-значением-—-лучшее-решение-для-количественных-переменных.-3.1.6"><span class="toc-item-num">3.1.6&nbsp;&nbsp;</span>Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.</a></span></li></ul></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

##  Откройте таблицу и изучите общую информацию о данных

### Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную data. Путь к файлу:

/datasets/data.csv

In [1]:
import pandas as pd
data=pd.read_csv('/datasets/data.csv')

<div class="alert alert-info">
<b>Комментарий студента №2</b>
    
Перенёс сюда загрузку библиотек для построения графиков
</div>

In [2]:
#загрузим библиотеки для визуализации, они нам понадобятся ближе к концу, когда будем строить графики
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

### Задание 2. Выведите первые 20 строчек датафрейма data на экран.

In [3]:
print(data.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

### Задание 3. Выведите основную информацию о датафрейме с помощью метода info().

In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


##  Предобработка данных¶

###  Удаление пропусков
#### Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.

In [5]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Данные в целом неплохие, пропуски по доходу и количенству дней работы объяснимы, но могут повлиять на наш результат.

#### Задание 5. В двух столбцах есть пропущенные значения. 
Один из них — days_employed. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
#Посчитаем сначала медианные значения дохода по каждой категории занятых.
med_income=data.groupby(['income_type'])['total_income'].median()
#Теперь заменим пропуски в столюце с доходами на полученные медианы

for d in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип дохода (d) и есть пропуски в total_income 
    data.loc[(data['income_type'] == d) & (data['total_income'].isna()), 'total_income'] = med_income[d]
    # и записываем в них медианное значение total_income среди строк с текущим типом дохода (d)

# проверяем, что все пропуски заполнены
print(data['total_income'].isna().sum())

0


###  Обработка аномальных значений

#### Задание 6. В данных могут встречаться артефакты (аномалии)
— значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода abs().

In [7]:
data['days_employed'] = data['days_employed'].abs()
print(data[data['days_employed']<0])#выполним проверку, не осталось ли отрицательных трудовых стажей


Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


#### Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа days_employed в днях.

In [8]:
print(data.groupby(['income_type'])['days_employed'].median())

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


Бросается в глаза, что безработные и пенсионеры имеют "страж" больше тысячи лет, что резко выделяется на общем фоне. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

#### Задание 8. Выведите перечень уникальных значений столбца children.

In [9]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


Очень интересно, 20 детей выглядит как артефакт, а -1 это очень грустная история.

#### Задание 9. В столбце children есть два аномальных значения. 
Удалите строки, в которых встречаются такие аномальные значения из датафрейма data.

In [10]:
#drop rows that contain specific 'value' in 'column_name'
data = data[data.children != -1 ]
data = data[data.children != 20 ]
print(data['children'].unique())

[1 0 3 2 4 5]


#### Задание 10. 
Ещё раз выведите перечень уникальных значений столбца children, чтобы убедиться, что артефакты удалены.

In [11]:
print(data['children'].unique())

[1 0 3 2 4 5]


### Удаление пропусков (продолжение)
#### Задание 11. Заполните пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type.

In [12]:
#Посчитаем сначала медианные значения срока занятости по каждой категории занятых.
med_days_employed=data.groupby(['income_type'])['days_employed'].median()
#Теперь заменим пропуски в столюце с days_employed на полученные медианы

for d in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип дохода (d) и есть пропуски в days_employed 
    data.loc[(data['income_type'] == d) & (data['days_employed'].isna()), 'days_employed'] = med_days_employed[d]
    # и записываем в них медианное значение days_employed среди строк с текущим типом дохода (d)


#### Задание 12. Убедитесь, что все пропуски заполнены. 
Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.

In [13]:
# проверяем, что все пропуски заполнены
print(data['days_employed'].isna().sum())

0


###  Изменение типов данных
#### Задание 13. Замените вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [14]:
data['total_income']=data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  int64  
 11  purpose           21402 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


###  Обработка дубликатов
#### Задание 14. Обработайте неявные дубликаты в столбце education. 
В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.

In [15]:
data['education']=data['education'].str.lower()

#### Задание 15. Выведите на экран количество строк-дубликатов в данных. 
Если такие строки присутствуют, удалите их. Сбрасывать индексы после удаления строк дубликатов с помощью reset_index(drop=True) здесь не требуется.

In [16]:
# посчитайте дубликаты
data.duplicated().sum()

71

In [17]:
# удалите дубликаты
data=data.drop_duplicates()
data.duplicated().sum() #проверка, что ничего не осталось

0

### 2.6  Категоризация данных
#### Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме data столбец total_income_category с категориями:

0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.
Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'. Используйте собственную функцию с именем categorize_income() и метод apply().

In [18]:
def categorize_income(row):# создайте функцию categorize_income()
    income=row['total_income']
    if income>=0 and income<=30000:
        return "E"
    elif income>=30001 and income<=50000:
        return "D"
    elif income>=50001 and income<=200000:
        return "C"
    elif income>=200001 and income<=1000000:
        return "B"
    elif income>=1000001 :
        return "A"
    else:
        print('Incorrect income data in row:', income)
# примените функцию методом apply()
data['total_income_category'] = data.apply(categorize_income, axis=1)

#### Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца purpose.

In [19]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


#### Задание 18. Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category,

в который войдут следующие категории:

'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.
Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.

Используйте собственную функцию с именем categorize_purpose() и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию.

In [20]:
# создайте функцию categorize_purpose()
def categorize_purpose(row):# создайте функцию categorize_income()
    purp=row['purpose']
    if "авто" in purp:
        return "операции с автомобилем"
    elif "недви" in purp or "жиль" in purp:
        return "операции с недвижимостью"
    elif "свад" in purp:
        return "проведение свадьбы"
    elif "образ" in purp:
        return "получение образования"
    else:
        print('Incorrect income data in row:', purp)

# примените функцию методом apply()
data['purpose_category'] = data.apply(categorize_purpose, axis=1)

## Вторая часть
###  3. Исследуйте данные и ответьте на вопросы

<div class="alert alert-danger">
<b>Комментарий ревьюера №1</b> ❌ Замечание по всему разделу ответов на вопросы 
    
В сводной нужно выводить количество клиентов, они нужны для анализа полноты выборок
</div>

<div class="alert alert-info">
<b>Комментарий студента №3</b>
    
Исправил функцию создающую сводную таблицу, так чтобы количество клиентов тоже отображалось
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера №2</b> ✅
    
Более привычно сводная выглядит, когда расчетные параметры находятся в столбцах. Ниже приведу пример функции
</div>

In [21]:
# КОД РЕВЬЮВЕРА

import plotly.express as px

def pivot(data, index):
    
    pivot = data.pivot_table(index = index, values = 'debt', aggfunc = ['count', 'sum', 'mean']).reset_index()
    pivot.columns = [index, 'Кол-во клиентов', 'Кол-во должников', 'Доля невозврата']
    display(pivot.sort_values('Доля невозврата', ascending = False))
    
    
    
    fig = px.bar(pivot, x = index, y = 'Доля невозврата', color = 'Доля невозврата', title = 'Доля невозврата по ' + index)
    fig.show()
    

pivot(data, 'children')

,children,Кол-во клиентов,Кол-во должников,Доля невозврата
4,4,41,4,0.097561
2,2,2052,194,0.094542
1,1,4808,444,0.092346
3,3,330,27,0.081818
0,0,14091,1063,0.075438
5,5,9,0,0.000000


#### Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?
Чтобы ответить на этот вопрос мы построим график, который будет показывать как изменяется доля кредитов вышедших на просрочку при увеличении количества детей. Но для начала эту долю надо посчитать.

In [22]:
#Учитывая, что последующие задания будут однотипные, сделаем функцию, которая считает процент просрочки 
#по оперделённой категории, поступающей в качестве переменной.
def cat_check(col):
    col_crosstab=pd.crosstab(data['debt'], data[col])
    col_crosstab.loc['Total']= col_crosstab.sum(axis = 0)
    col_crosstab.loc['share_of_problems']= round((col_crosstab.loc[1]/col_crosstab.loc['Total'])*100,1)
    col_crosstab.loc['share_of_good']= round((col_crosstab.loc[0]/col_crosstab.loc['Total'])*100,1)
    return(col_crosstab)
    
    
kids_crosstab=cat_check('children')
display(kids_crosstab)

children,0,1,2,3,4,5
debt,,,,,,
0,13028.0,4364.0,1858.0,303.0,37.0,9.0
1,1063.0,444.0,194.0,27.0,4.0,0.0
Total,14091.0,4808.0,2052.0,330.0,41.0,9.0
share_of_problems,7.5,9.2,9.5,8.2,9.8,0.0
share_of_good,92.5,90.8,90.5,91.8,90.2,100.0


In [23]:
#А теперь создадим функцию, чтобы визуализировать эту табилцу в форме интерактивных барчартов
def create_barchart(x_nums,y_nums,chart_title,axis_title):
    # создаем traces для визуализации
    trace0 = go.Bar(
    x=x_nums,
    y=y_nums
    )


    # создаем массив с данными и задаем title для графика и осей в layout
    layout = {'title': chart_title, 
              'xaxis': {'title': axis_title},'yaxis': {'title': 'Доля проблемных долгов,%'}
    }

    # создаем объект Figure и визуализируем его
    fig = go.Figure(data=trace0, layout=layout)
    iplot(fig, show_link=False)

In [24]:
#визуализируем соотношение количества детей и неплатежей
create_barchart(kids_crosstab.columns,kids_crosstab.loc['share_of_problems'],
                'Зависимость между количеством детей и возвратом кредита в срок','Количество детей'
)

<div class="alert alert-warning">
<b>Комментарий ревьюера №1</b> ⚠️
    
Все библиотеки нужно импортировать в одной ячейке в начале всего кода, это соответствует стилистическому гайду <a href="https://peps.python.org/pep-0008/">pep 8</a>. Длинные строки лучше разбивать на несколько
</div>
<div class="alert alert-info">
<b>Комментарий студента №4</b>
    
Перенёс.
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> ✅
    
Построение графика также можно поместить в ту же функцию
</div>
<div class="alert alert-info">
<b>Комментарий студента №5</b>
    
Поместил.
</div>

**Вывод напрашивается сам собой** - когда у тебя есть дети, платить по кредиту становится сложнее, причём с каждым новым ребёнком - доля просрочки растёт, исключение составляют семьи с тремя детьми, но они не ломают тренда. **Количество заёмщиков с 4  и 5 детьми настолько ничтожно, что их рассмотрение непоказательно. Даже семей с 3 детьми на порядок меньше, чем менее многодетных семей, так что и их более низкий уровень просрочек не факт что показателен.**

<div class="alert alert-info">
<b>Комментарий студента №6</b>
    
Модифицировал комментарий по детям, выделил жирным.
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> ✅
    
Ок
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера №2</b> ✅
    
👍
</div>

#### Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?
Здесь мы будем действовать аналогично. Посчитаем долю неплательщиков по каждому типу семейного положения и посмотрим как это будет выглядеть на графике.

In [25]:
#посчитаем долю заёмщиков в каждой категории по семейному положению.
family_crosstab=cat_check('family_status')
display(family_crosstab)

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
debt,,,,,
0,2523.0,1105.0,888.0,3749.0,11334.0
1,273.0,84.0,63.0,385.0,927.0
Total,2796.0,1189.0,951.0,4134.0,12261.0
share_of_problems,9.8,7.1,6.6,9.3,7.6
share_of_good,90.2,92.9,93.4,90.7,92.4


In [26]:
#визуализируем соотношение семейного положения и неплатежей
create_barchart(family_crosstab.columns,family_crosstab.loc['share_of_problems'],
                'Зависимость между семейным положением и возвратом кредита в срок','Семейное положение'
)


**Здесь вывод более однозначный.**
Есть небольшой блок условно одиноких людей, оводовевших или находящихся в разводе. Видимо эти битые жизнью люди привыкли осторожно обходиться с кредитами и их просрочки крайне редки. Те у кого официальный брак или хуже того - гражданский брак - имеют высокий рейтинг просрочников. Ну а неженатые/незамужние - абсолютные лидеры по просрочке. Вырисовывается такая картина, что только пройдя через горнило семейной жизни человек становится хорошим заёмщиком. **Но, есть также вероятность что тут больше влияет возраст, чем семейное положение, хорошая мысль для последующего более глубокого анализа.**

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> ✅
    
Хорошо
</div>

#### Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Тут нам пригодятся категории дохода, которые мы присвоили ранее. Давайте посмторим как по ним распределяется процент неплательщиков.

In [27]:
#посчитаем долю заёмщиков в каждой категории по доходам.
income_crosstab=cat_check('total_income_category')
display(income_crosstab)

total_income_category,A,B,C,D,E
debt,,,,,
0,23.0,4660.0,14568.0,328.0,20.0
1,2.0,354.0,1353.0,21.0,2.0
Total,25.0,5014.0,15921.0,349.0,22.0
share_of_problems,8.0,7.1,8.5,6.0,9.1
share_of_good,92.0,92.9,91.5,94.0,90.9


In [28]:
#визуализируем соотношение дохода и неплатежей
create_barchart(income_crosstab.columns,income_crosstab.loc['share_of_problems'],
                'Зависимость между уровнем дохода и возвратом кредита в срок','Уровень дохода'
)


**Вывод** К сожалению, предоставленная нам выборка, не позволяет сделать чётких выводов по категориям дохода А, D и E, так как количество таких заёмщиков слишком мало, и выбросы оказывают слишком сильное влияние.
По категориям В и С всё очевидно, со снижением уровня дохода, растёт количество невозвратов.

<div class="alert alert-danger">
<b>Комментарий ревьюера №1</b> ❌
    
Здесь лучше делать выводы по двум самым крупным категориям - B и С. По остальным категориям данных недостаточно для корректных выводов, статпогрешность большая
</div>
<div class="alert alert-info">
<b>Комментарий студента №7</b>
    
Вывод полностью исправил с учётом вышесказанного.
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера №2</b> ✅
    
👍
</div>

#### Задание 22. Как разные цели кредита влияют на его возврат в срок?

In [29]:
#посчитаем долю заёмщиков в каждой категории целей.
purpose_crosstab=cat_check('purpose_category')
display(purpose_crosstab)

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,,,,
0,3879.0,9971.0,3619.0,2130.0
1,400.0,780.0,369.0,183.0
Total,4279.0,10751.0,3988.0,2313.0
share_of_problems,9.3,7.3,9.3,7.9
share_of_good,90.7,92.7,90.7,92.1


In [30]:
#для наглядности отсортируем по возрастанию уровня просрочки
purpose_crosstab=purpose_crosstab.T
purpose_crosstab=purpose_crosstab.sort_values('share_of_problems', ascending=False)
purpose_crosstab=purpose_crosstab.T
#визуализируем соотношение целевого использования и неплатежей
create_barchart(purpose_crosstab.columns,purpose_crosstab.loc['share_of_problems'],
                'Зависимость между целевым использованием кредита и возвратом кредита в срок','Уровень дохода'
)


**Вывод** невооруженным глазом видно два кластера - более рискованные цели и менее рискованные. Риск в операциях с автомобилем высок по причине того, что всё что связано с машинами это постоянно растущая статья затрат, и на кредит может не хватить. С образованием ещё грустнее - по статистике лишь 45% поступивших в ВУЗ заканчивают обучение, и уж тем более нет никакой корреляции с академической успешностью и уровнем дохода. Отсюда и проблемы с кредитами на образование. Свадьба и операции с недвижимостью - тесно коррелируют с российской системой ценностей - создать семью, построить дом, посадить дерево... Заёмщики взявшие ипотеку, будут до последнего расплачиваться, чтобы не оказаться зимой на улице, а на свадьбу обычно дарят деньги, которые частно направляются на погашение кредита, вдобавок при умелом ведении хозяйства в рамках семьи, расходы на 1 члена ниже, чем у двух одиночек. Это имеет внутреннюю логику. Для банка важно, что проценты по кредитам на свадьбы и недвижку можно делать пониже, а по автокредам и кредитам на образование, ставочку лучше приподнять. Ну и к оценке заёмщиков по этой целёвке подходить более взвешенно.

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> ✅
    
Верно
</div>

#### Задание 23. Приведите возможные причины появления пропусков в исходных данных.

Традиционными причинами, приводящими к появлению пропусков, являются:
- невозможность получения или обработки данных;
- искажение или сокрытие информации;
- всевозможные поломки технического оборудования;
- природные явления;
- экономические причины (слишком дорого).

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> ✅
    
Возможно, эту информацию не требуют в некоторых случаях. Проблемы могли возникнуть на этапе загрузки и на этапе выгрузки данных. Неправильный запрос, объединение таблиц также может быть причиной.
</div>

#### Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Медиана -лучшее решение, так как:
- на нее не влияют случайные выбросы и шум;
- на неё не влияют пропуски данных;
- если данные имеют нормальное распределение и их количество велико, то медиана будет равняться среднему значению.

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> ✅
    
👍
</div>

## Общий вывод

- 1) Качество данных, предоставленных для анализа можно оценить как "хорошее", пропуски и дубликаты не встречаются в угрожающих количествах, доля "мусорных" данных, способных вызвать критическую ошибку в обработке минимальна.
- 2) Ни один из предложенных факторов не даёт высокой корреляции с возвратом кредита, и не может быть использован единолично для принятия кредитного решения.
- 3) Проведённый анализ данных позволяет выделить группу факторов повышающих кредитный риск: наличие детей (уже сам факт повышает процент неплатежей минимум на 0,7%), семейное положение отличное от "разведен/вдовец", а если "не замужем/холост" - то значительный прирост кредитного риска. Низкий уровень дохода - тоже можно рассматривать как риск-фактор, ну и наконец, цель кредита - операции с автомобилями или с образованием - тоже повышают кредитный риск.
- 4) В качестве митигантов кредитного риска можно признать факторы: отсутствие детей, семейное положение: разведён/вдовец, более выхокий доход  и цель кредита - свадьба или операции с недвижимостью.
- 5) Одетльно следует сказать о доходе - предоставленных данных по категориям с самым высоким и самым низким доходом, недостаточно, чтобы оценить уровень просрочки с достаточно низкой статистической погрешностью. Необходимо расширить выборку для получения более тончых результатов.

**Справочно: Над данными были проведены следующие операции:**
- 1) Были выявлены пропуске в столбце со сведениями о доходах. Они были заменены медианными значениями по каждому типу занятости.
- 2) Были вявлены пропуски в столбце с данными о количестве дней занятости. Некоторые значения почему-то оказались отрицательными, они были заменены на такие же, но положительные. Ещё бросается в глаза, что безработные и пенсионеры имеют "страж" больше тысячи лет, что резко выделяется на общем фоне. Эти артефакты не исправлялись, так как не использовались в дальнейших вычислениях.
- 3) Встречалось анамальное количество детей (-1) и 20, данные строки были исключены из анализа.
- 4) В столбце с образованием имелись дубликаты, различавшиеся только регистром. Они были исключены из анализа.
- 5) Была произведена категоризация данных о дохода по предложенной шкале и целей использования кредита по укрупнённым группировкам по смыслу.

**Рекомендации закачику:**
- Расширить выборку.
- Выяснить почему некорректно отражается трудовой стаж у пенсионеров и безработных.
- Выяснить причины пропусков в столбце с величиной доходов.
- Составить таблицы со словарями целевого использования и уровня образования.

<div class="alert alert-danger">
<b>Комментарий ревьюера №1</b> ❌ 
    
Можно пересмотреть выводы по доходу. Здесь также нужно описать какая работа над данными была проведена и добавить рекомендации банку-заказчику
</div>

<div class="alert alert-info">
<b>Комментарий студента №1</b>
    
Вывод был модифицирован с учётом комментарием ревьюера.
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера №2</b> ✅
    
По доходу можно попробовать разбить клиентов на равные по количеству человек группы. Для категоризации данных также можно использовать методы pd.cut() и pd.qcut(). Подробнее о них можно почитать здесь https://www.geeksforgeeks.org/how-to-use-pandas-cut-and-qcut/ или <a href="https://dfedorov.spb.ru/pandas/%D0%A0%D0%B0%D0%B7%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B2%20Pandas%20%D1%81%20%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E%20qcut%20%D0%B8%20cut.html">здесь</a>
</div>

<div style="border:solid purple 2px; padding: 20px"> 
<b> <font color='purple' >   
    
В целом, работа хорошая. У тебя отличные аналитические навыки, но остались небольшие доработки. Нужно добавить количества клиентов в сводные, проверить выводы по доходу и общий вывод. Доделывай и присылай на повторную проверку.
        
Жду твоих исправлений 😊
</font></b>
</div>

<div style="border:solid purple 2px; padding: 20px"> 
<b> <font color='purple' >   
Отлично! Критических замечаний больше нет. Проект принят! 👏 Поздравляю и желаю удачи!
</font></b>
</div>

<b>Привет!</b>

Меня зовут Харочкина Екатерина, и я буду проверять твой проект. Давай общаться на «ты», но если это неудобно, пожалуйста, сообщи. Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.

Ты подготовил замечательный проект, но есть аспекты, которые еще можно улучшить. Я буду использовать комментарии с цветовой разметкой. Пожалуйста, **не перемещай, не изменяй и не удаляй их**. Это поможет выполнить повторную проверку твоего проекта оперативнее.

<div class="alert alert-info">
<b>Комментарий студента №0</b>
    
Уважаемая Екатерина! Спасибо большое за столь скорый и информативный анализ моего проекта. Твои комментарии очень ценны для меня, постарался всё исправить в соответствии с ними.
</div>


<div class="alert alert-danger">
<b>Комментарий ревьюера №1</b> ❌
    
Таким комментарием выделены самые важные замечания. Без их отработки проект не будет принят.
</div>

<div class="alert alert-warning">
<b>Комментарий ревьюера №1</b> ⚠️
    
Таким комментарием выделены небольшие замечания и рекомендации, которые будет хорошо учесть.
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> ✅
    
Таким комментарием выделены удачные решения🙂
</div>

Свои комментарии ты можешь помечать синим цветом.

<div class="alert alert-info">
<b>Комментарий студента №1</b>
    
Так ты можешь оформить свой комментарий
</div>

Давай работать над проектом в диалоге: если ты **что-то меняешь** в проекте по моим рекомендациям — **пиши об этом**. После того, как ты все доработаешь, я смогу принять твой проект!) 